In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Data Cleaning


## Dataset 1: IMDB - Principles

Loading and checking the data:

In [2]:
prin = pd.read_csv('zippedData/imdb.title.principals.csv.gz')
prin.head(12)

,tconst,ordering,nconst,category,job,characters
0,tt0111414,1,nm0246005,actor,NaN,"[""The Man""]"
1,tt0111414,2,nm0398271,director,NaN,NaN
2,tt0111414,3,nm3739909,producer,producer,NaN
3,tt0323808,10,nm0059247,editor,NaN,NaN
4,tt0323808,1,nm3579312,actress,NaN,"[""Beth Boothby""]"
5,tt0323808,2,nm2694680,actor,NaN,"[""Steve Thomson""]"
6,tt0323808,3,nm0574615,actor,NaN,"[""Sir Lachlan Morrison""]"
7,tt0323808,4,nm0502652,actress,NaN,"[""Lady Delia Morrison""]"
8,tt0323808,5,nm0362736,director,NaN,NaN
9,tt0323808,6,nm0811056,producer,producer,NaN


In [3]:
prin.ordering = prin.ordering.apply(lambda x: int(x))

In [4]:
prin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1028186 entries, 0 to 1028185
Data columns (total 6 columns):
tconst        1028186 non-null object
ordering      1028186 non-null int64
nconst        1028186 non-null object
category      1028186 non-null object
job           177684 non-null object
characters    393360 non-null object
dtypes: int64(1), object(5)
memory usage: 47.1+ MB


Columns with null-values: Jobs, Characters  

The jobs that are in the jobs column seem match the the corresponding categories in the category column. Let's see if this is the case for all:

In [5]:
print(len(prin[(prin.category != prin.job) & prin.job.notna()].head()))
prin[(prin.category != prin.job) & prin.job.notna()].head()

5


,tconst,ordering,nconst,category,job,characters
19,tt0417610,6,nm0083201,writer,story,NaN
36,tt0475290,10,nm0005683,cinematographer,director of photography,NaN
46,tt0477302,10,nm0579580,cinematographer,director of photography,NaN
52,tt0477302,6,nm0744839,writer,screenplay,NaN
53,tt0477302,7,nm1583636,writer,novel,NaN


Removing the job column only affects 5 rows (out of  >1 million)

In [6]:
prin.drop(columns='job', inplace = True)

In [7]:
len(prin[prin.category.isin(['actor', 'actress'])])

402926

At first glance we see the characters column is more than half Null, but looking closer, we find that most actors have an assoiciated character: 393360 out of 402926. (Even if 15% are Him/herself)

In [8]:
prin.characters.value_counts().head()

["Himself"]     43584
["Herself"]     16127
["Narrator"]     2218
["Alex"]          656
["David"]         620
Name: characters, dtype: int64

Let's clean up those character names:

In [9]:
prin.characters =prin.characters.apply(lambda x: x.strip('["]') if isinstance(x, str) else x)

In [10]:
# prin.tconst.value_counts()
# prin.nconst.value_counts()
prin.ordering.value_counts()

1     143454
2     134649
3     126538
4     117775
5     108862
6     100140
7      90820
8      80587
9      69218
10     56143
Name: ordering, dtype: int64

In [11]:
prin.ordering[prin.category =='director'].unique()

array([ 2,  5,  3,  6,  1,  4,  7, 10,  8,  9], dtype=int64)

-No connection between ordering and category

I'm tempted to convert the dataset to one line per film, but I think it suits our needs as it is

Save clean dataset to file:

In [12]:
prin.to_csv('clean_data/clean_principles.csv', index = False)

## Dataset 2: IMDB - title basics

Loading and checking data

In [13]:
bas = pd.read_csv('zippedData/imdb.title.basics.csv.gz')

In [14]:
bas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
tconst             146144 non-null object
primary_title      146144 non-null object
original_title     146123 non-null object
start_year         146144 non-null int64
runtime_minutes    114405 non-null float64
genres             140736 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [15]:
bas.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


Checking each column, looking for e.g. placeholder values

In [16]:
bas.primary_title.value_counts().head()
bas.original_title.value_counts().head()
bas.start_year.value_counts().head()
bas.runtime_minutes.value_counts().head()

90.0     7131
80.0     3526
85.0     2915
100.0    2662
95.0     2549
Name: runtime_minutes, dtype: int64

In [17]:
#fill null 'original_title' entries with the primary title
bas.loc[(bas.original_title.isna()),
        'original_title'] = bas.loc[(bas.original_title.isna()),
                                    'primary_title']

In [18]:
bas.describe()

,start_year,runtime_minutes
count,146144.000000,114405.000000
mean,2014.621798,86.187247
std,2.733583,166.360590
min,2010.000000,1.000000
25%,2012.000000,70.000000
50%,2015.000000,87.000000
75%,2017.000000,99.000000
max,2115.000000,51420.000000


In [19]:
bas[bas.start_year == 2115]
#Checked it out, "upcoming experimental science fiction film" -year is correct

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
89506,tt5174640,100 Years,100 Years,2115,NaN,Drama


In [20]:
#But will remove as an extreme outlier; MS probably shan't want to replicate it
bas.drop(89506, inplace = True)

In [21]:
bas.describe()

,start_year,runtime_minutes
count,146143.000000,114405.000000
mean,2014.621111,86.187247
std,2.720952,166.360590
min,2010.000000,1.000000
25%,2012.000000,70.000000
50%,2015.000000,87.000000
75%,2017.000000,99.000000
max,2027.000000,51420.000000


In [22]:
#replacing null runtimes with the median
bas.loc[bas.runtime_minutes.isna(), 'runtime_minutes'] =87.0

In [23]:
bas.describe()
#Note: lowered the std, and interquartile range but it's too many rows to drop

,start_year,runtime_minutes
count,146143.000000,146143.000000
mean,2014.621111,86.363753
std,2.720952,147.192154
min,2010.000000,1.000000
25%,2012.000000,75.000000
50%,2015.000000,87.000000
75%,2017.000000,95.000000
max,2027.000000,51420.000000


In [24]:
bas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146143 entries, 0 to 146143
Data columns (total 6 columns):
tconst             146143 non-null object
primary_title      146143 non-null object
original_title     146143 non-null object
start_year         146143 non-null int64
runtime_minutes    146143 non-null float64
genres             140735 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 7.8+ MB


Genres are something I'll being looking at and only a small percentage are missing, so I'll drop rows

In [25]:
#by this point the only null values are in the genres column
bas.dropna(inplace = True)

In [26]:
#create list of all genres
genres = []
for f in bas.genres:
    for g in f.split(','):
        if not g in genres:
            genres.append(g)

In [27]:
#create a column for each genre, give value True if the string appears 
#...in the original genre_ids column
for g in genres:
    bas[g] = bas.genres.apply(lambda x: g in x)

In [28]:
#Check it worked:
bas[['genres']+ genres].head()

,genres,Action,Crime,Drama,Biography,Comedy,Fantasy,Horror,Thriller,Adventure,Animation,Documentary,History,Mystery,Sci-Fi,Romance,Family,War,Music,Sport,Western,Musical,Adult,News,Talk-Show,Reality-TV,Game-Show,Short
0,"Action,Crime,Drama",True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,"Biography,Drama",False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,Drama,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,"Comedy,Drama",False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,"Comedy,Drama,Fantasy",False,False,True,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


Save cleaned data to file:

In [29]:
bas.to_csv('clean_data/clean_basics.csv', index = False)

## Dataset 3: IMDB - name

Loading and Checking data:

In [30]:
nam = pd.read_csv('zippedData/imdb.name.basics.csv.gz')

In [31]:
nam.head()

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer","tt1470654,tt0363631,tt0104030,tt0102898"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department","tt0114371,tt2004304,tt1618448,tt1224387"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator","tt0452644,tt0452692,tt3458030,tt2178256"


In [32]:
nam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606648 entries, 0 to 606647
Data columns (total 6 columns):
nconst                606648 non-null object
primary_name          606648 non-null object
birth_year            82736 non-null float64
death_year            6783 non-null float64
primary_profession    555308 non-null object
known_for_titles      576444 non-null object
dtypes: float64(2), object(4)
memory usage: 27.8+ MB


The main use we'll have for this dataset is just to attach names to other tables via nconst

In [33]:
nam.describe()

,birth_year,death_year
count,82736.000000,6783.000000
mean,1967.043826,2000.523367
std,22.122190,43.951530
min,1.000000,17.000000
25%,1957.000000,2001.000000
50%,1971.000000,2013.000000
75%,1981.000000,2016.000000
max,2014.000000,2019.000000


In [34]:
nam[nam.birth_year < 1650]

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
1244,nm0963344,John Bunyan,1628.0,1688.0,"writer,soundtrack","tt0393584,tt0194913,tt0234464,tt1336619"
11901,nm0549265,Christopher Marlowe,1564.0,1593.0,"writer,soundtrack","tt0114279,tt0062898,tt0287837,tt0101798"
21094,nm0613556,Shikibu Murasaki,973.0,1031.0,writer,"tt0092875,tt0043580,tt0094100,tt0256385"
38106,nm0674518,Charles Perrault,1628.0,1703.0,writer,"tt0053285,tt1587310,tt0042332,tt8651654"
39848,nm0596942,Molière,1622.0,1673.0,"writer,soundtrack,miscellaneous","tt0154249,tt0026069,tt0017448,tt0370853"
52088,nm1063158,Cheng'en Wu,1506.0,1581.0,writer,"tt1163129,tt6168860,tt0112778,tt1753783"
52523,nm0148859,Miguel de Cervantes y Saavedra,1547.0,1616.0,"writer,soundtrack","tt0204285,tt6151214,tt0023956,tt0039330"
59713,nm0000636,William Shakespeare,1564.0,1616.0,"writer,soundtrack,miscellaneous","tt8632012,tt3894536,tt5932378,tt7344870"
71418,nm0478548,Madame de La Fayette,1634.0,1693.0,writer,"tt1599975,tt0158117,tt0204761,tt1263778"
74627,nm0017266,Mariana Alcoforado,1640.0,1723.0,writer,"tt0062836,tt0076312,tt2219674"


As I go back looking for implausible birth years, the more I find they are correct (e.g. Murasaki Shikibu (紫 式部, English: Lady Murasaki; c. 973 or 978 – c. 1014 or 1031) was a Japanese novelist, poet and lady-in-waiting at the Imperial court during the Heian period. -Wikipedia). Except for Michael Vignola, born (Nov 1st) 1980

In [35]:
nam.loc[nam['birth_year'] == 1.0, 'birth_year'] = 1980.0

In [36]:
nam.death_year.value_counts().head()

2018.0    624
2017.0    621
2016.0    592
2015.0    540
2013.0    437
Name: death_year, dtype: int64

Despite only a small fraction having birth years, it's still more than 80k, a large enough number if to do something with, so I'll leave it as is. The number of notnull death years seems to be to small a fraction of the birth years to be complete. However with the skew in birth years towards the present (There are more famous film people now than previously), I'll give it the benefit of the doubt and leave it in.

For primary_profession and known_for_titles, I'll leave the null values as they are. Too many entries to drop the column, not worth dropping the rows for a column that might not be used.

In [37]:
# nam.primary_profession.value_counts()
# nam.known_for_titles.value_counts()

Save cleaned data to file:

In [38]:
nam.to_csv('clean_data/clean_names.csv', index = False)

## Dataset 4: TMDB -Movies
Loading and checking the data:

In [39]:
tmdb = pd.read_csv('zippedData/tmdb.movies.csv.gz')
tmdb.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [40]:
tmdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
Unnamed: 0           26517 non-null int64
genre_ids            26517 non-null object
id                   26517 non-null int64
original_language    26517 non-null object
original_title       26517 non-null object
popularity           26517 non-null float64
release_date         26517 non-null object
title                26517 non-null object
vote_average         26517 non-null float64
vote_count           26517 non-null int64
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


In [41]:
genre_dict = {'28':"Action", '12':"Adventure", '16':"Animation", '35':"Comedy", 
              '80':"Crime", '99':"Documentary", '18':"Drama", '10751':"Family", 
              '14':"Fantasy", '36':"History", '27':"Horror", '10402':"Music", 
              '9648':"Mystery", '10749':"Romance", '878':"Science Fiction", 
              '10770':"TV Movie", '53':"Thriller", '10752':"War",'37':"Western"}

In [42]:
#create list of all genres
genres = []
for f in tmdb.genre_ids:
    for g in f.strip('[]').split(','):
        if not g in genres:
            genres.append(g)

In [43]:
#clean list to remove multiples with only differing spaces
genres = set(list(map(lambda x: x.strip(' '), genres)))

In [44]:
#create a column for each genre, give value True if the string appears 
#...in the original genre_ids column
for g in genres:
    tmdb[g] = tmdb.genre_ids.apply(lambda x: g in x)

In [45]:
tmdb.head(2)

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count,18,,35,9648,53,36,10749,14,878,28,12,10751,10402,37,99,10770,16,80,10752,27
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788,False,True,False,False,False,False,False,True,False,False,True,True,False,False,False,False,False,False,False,False
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610,False,True,False,False,False,False,False,True,False,False,True,True,False,False,False,False,True,False,False,False


In [46]:
tmdb.drop(columns ='', inplace = True)

In [47]:
tmdb.rename(columns = genre_dict, inplace = True)

In [48]:
tmdb.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count,Drama,Comedy,Mystery,Thriller,History,Romance,Fantasy,Science Fiction,Action,Adventure,Family,Music,Western,Documentary,TV Movie,Animation,Crime,War,Horror
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788,False,False,False,False,False,False,True,False,False,True,True,False,False,False,False,False,False,False,False
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610,False,False,False,False,False,False,True,False,False,True,True,False,False,False,False,True,False,False,False
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368,False,False,False,False,False,False,False,True,True,True,False,False,False,False,False,False,False,False,False
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186,False,False,False,False,False,False,False,True,True,True,False,False,False,False,False,False,False,False,False


In [49]:
#Sort genre columns into alphabetical order:
tmdb = tmdb.iloc[:,:10].join(tmdb.iloc[:,10:].sort_index(axis=1))

In [50]:
#remove extra index column and the -no longer necessary- genre_ids column
tmdb.drop(columns = ['Unnamed: 0', 'genre_ids'], inplace = True)

In [51]:
tmdb.id.value_counts().head()

292086    3
463839    3
11976     3
391872    3
416572    3
Name: id, dtype: int64

In [52]:
tmdb[tmdb.id == 292086] #duplicate entries found

,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
5838,292086,en,Come Morning,3.013,2012-10-21,Come Morning,6.3,5,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False
11836,292086,en,Come Morning,3.013,2012-10-21,Come Morning,6.3,5,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False
21565,292086,en,Come Morning,3.013,2012-10-21,Come Morning,6.3,5,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False


In [53]:
tmdb.drop_duplicates(inplace = True)

In [54]:
#check for placeholders
tmdb.original_language.value_counts().head()
tmdb.original_title.value_counts().head()
tmdb.popularity.value_counts().head()
tmdb.release_date.value_counts().head()
tmdb.title.value_counts().head()
tmdb.vote_average.value_counts().head()
tmdb.vote_count.value_counts().head()

1    6438
2    2994
3    1720
4    1303
5     945
Name: vote_count, dtype: int64

Save cleaned data to file:

In [55]:
tmdb.to_csv('clean_data/clean_tmdb.cs', index = False)